In [1]:
# If an error occurs in this cell, it is likely because you need to pip install a package(s) 
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
import nltk
#nltk.download('wordnet')
#from nltk.corpus import stopwords
#from nltk.stem.lancaster import LancasterStemmer
import gensim.corpora as corpora
from pprint import pprint 
import ast
from glob import glob
from collections import Counter
import re
import matplotlib.pyplot as plt
import csv
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
from wordcloud import WordCloud 
import seaborn as sns

In [3]:
# Set directory and get data - may need to modify this to wherever you have the data stored 
#os.chdir("/Users/jordan/Desktop/NLP_Syllabi_Project_copy/data_and_output")
data = pd.read_csv("cleaned_data_new.csv", lineterminator='\n')
data.drop(data.columns[[0, 1, 2]], axis = 1, inplace = True) #drop meaningless indice columns 
data['index'] = data.index
#print(data.head())
#len(data.index) #4889 documents 

In [4]:
# Create new data, just text and index 
data = data[['Corpus', 'index']] 
#print(data.head())

In [5]:
# Further preprocessing (removes stop words and also words 3 characters or less) 
# Note: There was additional preprocessing done in previous steps (earlier scripts located in scripts folder)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
            result.append(token)
    return result

#doc_sample = data[data['index'] == 1].values[0][0]
#print('original document: ')
#words = []
#for word in doc_sample.split(' '):
#    words.append(word)
#print(words)
#print('\n\n tokenized and document: ')
#print(preprocess(doc_sample))

processed_text = data['Corpus'].astype('str').map(preprocess)
#print(processed_text[:10])

In [6]:
# Create a dictionary
dictionary = gensim.corpora.Dictionary(processed_text)
#count = 0 
#for k, v in dictionary.iteritems():
#    print(k, v)
#    count += 1
#    if count > 10:
#        break

In [7]:
# Interactive LDA modeling 
print("NOTE: CHANGING MODEL PARAMETERS WILL PRODUCE NEW OUTPUT AND MAY TAKE AWHILE TO REFRESH")
@interact 
def output(low_freq=(1, 50, 1), high_freq=(0.05, 0.7, 0.05), topic_num = (10, 100, 10)):
    dictionary.filter_extremes(no_below=low_freq, no_above = high_freq) #modify these 
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_num, id2word=dictionary) 
    top_words_per_topic = []
    for t in range(lda_model.num_topics):
        y = [str(t)]
        x = lda_model.get_topic_terms(t,10)
        y.extend([dictionary[pair[0]] for pair in x])
        top_words_per_topic.append(y)
        y=[]
    top_words_per_topic
    word_list = ["Word" + str(i) for i in list(range(1,11))]
    word_list
    word_list.insert(0, "Topic")
    out = pd.DataFrame(top_words_per_topic, columns=word_list)
    # This print statement is to help the viewer know when output has refreshed 
    print("You are viewing results from a model with the following paramaters: \n"  "Low frequency cutoff=" + str(low_freq) + "\n" + 
          "High frequency cutoff="+str(round(high_freq, 2)) + "\n" + "Number of topics=" + str(topic_num))
    return out

NOTE: CHANGING MODEL PARAMETERS WILL PRODUCE NEW OUTPUT AND MAY TAKE AWHILE TO REFRESH


interactive(children=(IntSlider(value=25, description='low_freq', max=50, min=1), FloatSlider(value=0.35000000…